In [19]:
import numpy as np

class ExpRandom:
  def __init__(self, lam):
    self.lam = lam
    
  def generate(self):
    return ((-np.log(1-(np.random.uniform(low=0.0,high=1.0)))) / self.lam)

class HypoExpRandom:
  def __init__(self, lam1, lam2):
    self.lam1 = lam1
    self.lam2 = lam2
    x = (np.log(lam1) - np.log(lam2)) / (lam1 - lam2)
    self.maxy = self.__calculate(x)

    increment = x/100
    while self.__cumulative(x) < 0.999:
      x += increment
      
    self.maxx = x
  
  def generate(self):
    while True:
      x = np.random.uniform(low=0.0,high=self.maxx)
      y = np.random.uniform(low=0.0,high=self.maxy)
      if y <= self.__calculate(x):
        return x
  
  def __calculate(self, x):
    return ((self.lam1*self.lam2) / (self.lam1 - self.lam2)) * (np.exp(-x*self.lam2) - np.exp(-x*self.lam1))
  
  def __cumulative(self, x):
    return 1 - (self.lam2 / (self.lam2 - self.lam1)) * np.exp(-x*self.lam1) + (self.lam1 / (self.lam2 - self.lam1)) * np.exp(-x*self.lam2)

class Event:
  ARRIVAL = 0
  DEPARTURE = 1
  SIM_END = 2
  def __init__(self, event_type, time):
    self.event_type = event_type
    self.time = time
  
  def __str__(self):
    type_s = ""
    if self.event_type == self.ARRIVAL:
      type_s = "ARRIVAL"
    elif self.event_type == self.DEPARTURE:
      type_s = "DEPARTURE"
    elif self.event_type == self.SIM_END:
      type_s = "SIM_END"
    
    return "Type: {0}, Time: {1}".format(type_s, self.time)

  def __repr__(self):
    type_s = ""
    if self.event_type == self.ARRIVAL:
      type_s = "ARRIVAL"
    elif self.event_type == self.DEPARTURE:
      type_s = "DEPARTURE"
    elif self.event_type == self.SIM_END:
      type_s = "SIM_END"
    
    return "<Event Type: {0}, Time: {1}>".format(type_s, self.time)


class EventList:
  def __init__(self):
    self.event_list = []
  
  def schedule(self, event):
    if len(self.event_list) == 0:
      self.event_list.append(event)
    else:
      for index, ev in enumerate(self.event_list):
        if ev.time > event.time:
          self.event_list.insert(index, event)
          break
        elif index == len(self.event_list) - 1:
          self.event_list.append(event)
          break
  
  def get(self):
    if len(self.event_list) == 0:
      return None
    else:
      next_event = self.event_list[0]
      del self.event_list[0]
      return next_event

  def __str__(self):
    return str(self.event_list)

class MH1Simulator:
  def __init__(self, lam, mu1, mu2, sim_end_time = 15000):
    self.history = []

    self.running = False
    self.sim_end_time = sim_end_time

    self.arrivals = 0
    self.departures = 0
    self.current_jobs = 0

    self.old_time = 0
    self.area = 0
    self.btime = 0
    self.current_time = 0

    self.nmean = 0
    self.U = 0
    self.wmean = 0
    self.throughput = 0

    self.lam = lam
    self.mu1 = mu1
    self.mu2 = mu2

    self.exp_generator = ExpRandom(lam)
    self.hypo_exp_generator = HypoExpRandom(mu1, mu2)

    self.event_list = EventList()
    self.event_list.schedule(Event(Event.ARRIVAL, 0))
    self.event_list.schedule(Event(Event.SIM_END, sim_end_time))

  def start(self):
    self.running = True
    while self.running:
      self.__clock()
    print("Fine della simulazione")

  def __clock(self):
    event = self.event_list.get()
    self.current_time = event.time
    if event.event_type == Event.ARRIVAL:
      self.history.append([Event(Event.ARRIVAL, event.time), None])
      self.__arrival()
    elif event.event_type == Event.DEPARTURE:
      self.history[self.departures][1] = Event(Event.DEPARTURE, event.time)
      self.__departure()
    elif event.event_type == Event.SIM_END:
      self.__sim_end()
      self.running = False

  def __arrival(self):
    self.__stat_accumulator()
    self.current_jobs += 1
    self.arrivals += 1
    next_arrival_time = self.exp_generator.generate()
    ev = Event(Event.ARRIVAL, self.current_time + next_arrival_time)
    self.event_list.schedule(ev)

    if self.current_jobs == 1:
      next_departure_time = self.hypo_exp_generator.generate()
      ev = Event(Event.DEPARTURE, self.current_time + next_departure_time)
      self.event_list.schedule(ev)
  
  def __departure(self):
    self.__stat_accumulator()
    self.current_jobs -= 1
    self.departures += 1
    if self.current_jobs > 0:
      next_departure_time = self.hypo_exp_generator.generate()
      ev = Event(Event.DEPARTURE, self.current_time + next_departure_time)
      self.event_list.schedule(ev)
  
  def __sim_end(self):
    self.nmean = self.area / self.current_time
    self.wmean = self.area / self.departures
    self.throughput = self.departures / self.current_time
    self.U = self.btime / self.current_time

    print("Risultati della simulazione")
    print("Arrivi: {0}".format(self.arrivals))
    print("Partenze: {0}".format(self.departures))
    print("Throughput: {0}".format(self.throughput))
    print("Utilizzazione: {0}".format(self.U))
    print("Numero medio di clienti nel sistema: {0}".format(self.nmean))
    print("Tempo medio di permanenza nel sistema: {0}".format(self.wmean))
  
  def __stat_accumulator(self):
    interval = self.current_time - self.old_time
    self.old_time = self.current_time
    if self.current_jobs > 0:
      self.area += interval * self.current_jobs
      self.btime += interval

In [43]:
simulation = MH1Simulator(0.26, 1.09, 0.75)
simulation.start()


Risultati della simulazione
Arrivi: 3849
Partenze: 3848
Throughput: 0.25653333333333334
Utilizzazione: 0.573044839883828
Numero medio di clienti nel sistema: 1.1790842010030833
Tempo medio di permanenza nel sistema: 4.596222197257341
Fine della simulazione


In [47]:
print(simulation.history[0])

[<Event Type: ARRIVAL, Time: 0>, <Event Type: DEPARTURE, Time: 0.9766860773530284>]


In [48]:
import csv

with open("data.csv", "w") as f:
  writer = csv.writer(f)
  for index, el in enumerate(simulation.history):
    if el[0] is not None and el[1] is not None:
      writer.writerow([index+1, el[0].time, el[1].time, el[1].time - el[0].time])

In [3]:
simulation = MH1Simulator(0.13, 1.09, 0.75)
simulation.start()

Risultati della simulazione
Arrivi: 1950
Partenze: 1949
Throughput: 0.12993333333333335
Utilizzazione: 0.2875722608593525
Numero medio di clienti nel sistema: 0.37118251861161194
Tempo medio di permanenza nel sistema: 2.856715125281775
Fine della simulazione


In [24]:
simulation = MH1Simulator(0.09, 1.09, 0.75, sim_end_time = 300000)
simulation.start()

Risultati della simulazione
Arrivi: 26870
Partenze: 26870
Throughput: 0.08956666666666667
Utilizzazione: 0.20183591196145653
Numero medio di clienti nel sistema: 0.23993060174577555
Tempo medio di permanenza nel sistema: 2.678793469435529
Fine della simulazione
